# Como o uso de RAG melhora a contextualização?
## Técnicas Rag para o melhorar o Few-shot

Import de bibliotecas, leitura da chave da API e do arquivo txt usado para o RAG:

In [102]:
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.load import dumps, loads
from IPython.display import display, Markdown

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# read txt file
with open("brasil.txt", "r", encoding="utf-8") as file:
    conteudo = file.read()

Separação do textos (chunks) para o processo de embedding:

In [72]:
texts = conteudo.split("\n\n")
texts = [text for text in texts if text]
texts

['1. **Povos Indígenas e Período Pré-Colonial:** Antes da chegada dos europeus em 1500, o território que hoje conhecemos como Brasil era um mosaico vibrante de culturas e sociedades indígenas.  Estima-se que até 5 milhões de pessoas, distribuídas em centenas de diferentes etnias, habitavam a região, cada uma com sua própria língua, costumes e organização social.  Grupos como os Tupi-Guarani, os Jê e os Arawak, entre muitos outros,  desenvolveram complexas relações com o ambiente,  dominando técnicas agrícolas adaptadas aos diversos ecossistemas brasileiros,  além da caça, pesca e coleta.  Sua cosmovisão, profundamente ligada à natureza,  influenciava todos os aspectos da vida, desde a organização social e política até as práticas religiosas e artísticas.  Longe da imagem estereotipada de um povo homogêneo, os indígenas pré-colombianos demonstravam uma rica diversidade cultural e um profundo conhecimento do território,  legado que, apesar dos séculos de violência e desrespeito, ainda re

Embedding dos textos e criação do Vector Database:

In [89]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

faiss = FAISS.from_texts(texts, embeddings)

# salva o arquivo
faiss.save_local("vectorstore/docs_index")

Carregamento do Vector Database e criação do retriever:

In [90]:
vector_store = FAISS.load_local(
    "vectorstore/docs_index", embeddings, allow_dangerous_deserialization=True
)

# Busca de dois documentos
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002955F199BE0>, search_kwargs={'k': 3})

Execução da busca de documentos usando apenas a base do RAG:

In [91]:
query = "Quais foram as causas da Proclamação da República no Brasil?"

# busca
response1 = retriever.invoke(query)
for i in response1:
    print(i.page_content)

12. **Proclamação da República (1889):**  O golpe militar que instaurou a República em 15 de novembro de 1889 marcou uma mudança radical no regime político brasileiro, encerrando o período imperial e dando início à chamada República Velha.  Apesar da promessa de modernização e progresso, a República Velha foi marcada pela oligarquia cafeeira e pela política do café com leite, na qual o poder se alternava entre as elites de São Paulo e Minas Gerais.  Esse sistema excluía grande parte da população da participação política e perpetuava as desigualdades sociais herdadas do período imperial. A proclamação, embora celebrada como um avanço, na prática representou a troca de uma elite dominante por outra, com poucas mudanças estruturais na sociedade brasileira.
8. **Proclamação da Independência (1822):**  Após anos de crescente  autonomia  e  tensões com as Cortes Portuguesas, que desejavam recolonizar o Brasil,  D. Pedro I proclamou a independência do país em 7 de setembro de 1822.  Esse ato,

Função para fazer o re-ranqueamento dos documentos para RAG-Fusion:

In [93]:
# https://github.com/langchain-ai/rag-from-scratch/tree/main

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents
        and an optional parameter k used in the RRF formula """

    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    
    # select the top 3
    reranked_results = reranked_results[:3]

    return reranked_results

Chain para executar o RAG Fusion:

In [94]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

template_rag = """Você é um assistente de modelo de linguagem de IA. Sua tarefa é gerar quatro versões diferentes da pergunta fornecida pelo usuário para recuperar
documentos relevantes de um banco de dados vetorial. Ao gerar várias perspectivas sobre a pergunta do usuário de contexto semântico idêntico,
seu objetivo é ajudar o usuário a superar algumas das limitações da busca de similaridade baseada em distância, mantendo a intenção original da busca como causa e consequência.
Forneça essas perguntas alternativas separadas por novas linhas. \n
Gere novas perguntas relacionadas a: {question} \n
Excreva apenas as perguntas, sem formatação adicional. \n
Saída (3 consultas):"""


prompt_rag_fusion = PromptTemplate.from_template(template_rag)

generate_queries = (
    prompt_rag_fusion
    | llm
    | StrOutputParser()
    | (lambda x: [line.strip() for line in x.split("\n") if line.strip()])
)

#Chain com RRF
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion

In [113]:
generate_queries.invoke(query)

['Quais fatores contribuíram para a queda da monarquia brasileira e a subsequente proclamação da república?',
 'Quais eventos e circunstâncias levaram à Proclamação da República no Brasil em 1889?',
 'Quais foram os motivos políticos, sociais e econômicos que resultaram na proclamação da República no Brasil?',
 'Como o contexto sociopolítico do Brasil no final do século XIX propiciou a Proclamação da República?']

Execução da busca de documentos usando o RAG-Fusion:

In [95]:
response2 = retrieval_chain_rag_fusion.invoke(query)
for i in response2:
    print(i[0].page_content)

12. **Proclamação da República (1889):**  O golpe militar que instaurou a República em 15 de novembro de 1889 marcou uma mudança radical no regime político brasileiro, encerrando o período imperial e dando início à chamada República Velha.  Apesar da promessa de modernização e progresso, a República Velha foi marcada pela oligarquia cafeeira e pela política do café com leite, na qual o poder se alternava entre as elites de São Paulo e Minas Gerais.  Esse sistema excluía grande parte da população da participação política e perpetuava as desigualdades sociais herdadas do período imperial. A proclamação, embora celebrada como um avanço, na prática representou a troca de uma elite dominante por outra, com poucas mudanças estruturais na sociedade brasileira.
13. **Primeira República e Tensão Social (1889–1930):** A Primeira República, também conhecida como República Velha, foi um período de intensas contradições.  Por um lado, testemunhou o início da industrialização e o crescimento das cid

In [107]:
def format_docs_rag(docs):
    return "\n\n".join([d.page_content for d in docs])

# Prompt de geração com Few-Shot Dinâmico
sql_generation_prompt_template = """
Você é uma ferramenta desenvolvida para responder perguntas do usuário sobre a história do Brasil.
Responda no formato de Markdown.
Use somente as informações fornecidas nos documentos para responder à pergunta.
Os documentos de referência são:
{context}

Pergunta: {question}
"""

main_prompt = PromptTemplate.from_template(sql_generation_prompt_template)

# Chain para gerar a query SQL
main_chain_rag = (
    {
        "context": retriever | format_docs_rag,
        "question": RunnablePassthrough()
    }
    | main_prompt
    | llm
    | StrOutputParser()
)

In [108]:
def format_docs_rag_fusion(docs):
    return "\n\n".join([d[0].page_content for d in docs])

# Prompt de geração com Few-Shot Dinâmico
sql_generation_prompt_template = """
Você é uma ferramenta desenvolvida para responder perguntas do usuário sobre a história do Brasil.
Responda no formato de Markdown.
Use somente as informações fornecidas nos documentos para responder à pergunta.
Os documentos de referência são:
{context}

Pergunta: {question}
"""

main_prompt = PromptTemplate.from_template(sql_generation_prompt_template)

# Chain para gerar a query SQL
main_chain_rag_fusion = (
    {
        "context": retrieval_chain_rag_fusion | format_docs_rag_fusion,
        "question": RunnablePassthrough()
    }
    | main_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
display(Markdown(main_chain_rag.invoke(query)))

O documento 12 menciona que a Proclamação da República, em 15 de novembro de 1889, foi resultado de um golpe militar.  Embora prometesse modernização e progresso, a República Velha foi caracterizada pela oligarquia cafeeira e pela política do "café com leite",  demonstrando que a mudança representou uma troca de elites dominantes, com poucas mudanças estruturais.  O documento não detalha outras causas específicas para o golpe.


In [110]:
display(Markdown(main_chain_rag_fusion.invoke(query)))

A Proclamação da República, em 15 de novembro de 1889, foi resultado de um golpe militar que encerrou o período imperial.  Embora a abolição da escravatura em 1888 tenha sido um marco fundamental, a falta de políticas de inclusão social para os libertos gerou um ciclo de pobreza e marginalização, e enfraqueceu o apoio da elite agrária (que dependia da escravidão) à monarquia.  Esse descontentamento contribuiu para o clima de instabilidade que culminou na proclamação da República.  A própria República Velha, apesar da promessa de modernização, foi marcada pela oligarquia cafeeira e pela política do "café com leite", excluindo grande parte da população da participação política e perpetuando desigualdades.
